In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


import pandas as pd
import pyodbc
import time
import threading
import tkinter as tk
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Variável global para controlar se o loop deve continuar
deve_continuar = True

# Configuração da fonte de dados ODBC (DSN)
nome_dsn = 'teste'  # Substitua pelo nome da sua DSN configurada
usuario = 'root'
senha = '@senha123'
banco_de_dados = 'teste'  # Nome do seu banco de dados

# String de conexão ODBC
string_conexao_odbc = f'DSN={nome_dsn};UID={usuario};PWD={senha};DATABASE={banco_de_dados}'

# Consulta SQL 
consulta_sql = f"""
SELECT ped_spvcodigo AS serie_pedido, ped_numero AS numero_pedido, ped_stpcodigo AS cod_separacao, ped_dtemissao AS data_emissao, ped_dtentrega AS data_entrega
FROM pedidos
WHERE ped_stpcodigo IN (2, 8)
AND ped_dtemissao <= CURRENT_DATE
AND ped_dtentrega <= CURRENT_DATE
ORDER BY ped_dtemissao ASC, ped_dtentrega ASC;
"""

# Conectar ao banco de dados MySQL via ODBC
conexao = pyodbc.connect(string_conexao_odbc)

# Executar a consulta SQL e carregar os resultados em um DataFrame do Pandas
df = pd.read_sql(consulta_sql, conexao)

# Fechar a conexão com o banco de dados
conexao.close()

# Atualize o painel de pedidos
pedidos_nao_separados = df[(df['cod_separacao'] == 2) | (df['cod_separacao'] == 8)]
pedidos_ordenados = pedidos_nao_separados.sort_values(by=['data_emissao', 'data_entrega'])
pedidos_painel = pedidos_ordenados.head(8)

# Função para exibir o painel
def exibir_painel(janela, frame, pedidos):
    # Limpa o frame existente
    for widget in frame.winfo_children():
        widget.destroy()

    # Cria 9 retângulos (labels) e os adiciona ao frame
    for i in range(9):
        if i < len(pedidos):
            # Concatena serie_pedido e numero_pedido com um hífen entre eles
            pedido = f"{pedidos.iloc[i]['serie_pedido']}-{pedidos.iloc[i]['numero_pedido']}"
            
            # Substitui o status 2 por 'Não Separado' e 8 por 'Parcial'
            status = 'Não Separado' if pedidos.iloc[i]['cod_separacao'] == 2 else 'Parcial'
            
            # Define a cor de fundo com base no status
            cor_fundo = "lightyellow" if status == 'Parcial' else "lightblue"
            
            texto = f"Pedido: {pedido}\nStatus: {status}"
        else:
            texto = "Aguardando Novo Pedido"
        
        label = tk.Label(frame, text=texto, borderwidth=2, relief="solid", font=("Arial", 24), bg=cor_fundo, fg="black", width=20, height=5)  # Melhora a aparência dos retângulos
        label.grid(row=i//3, column=i%3, padx=20, pady=20)

    # Agenda a próxima atualização para daqui a 30 segundos
    janela.after(30000, lambda: atualizar_painel(janela, frame))


# Função para atualizar o painel
def atualizar_painel(janela, frame):
    # Refaz a consulta ao banco de dados e atualize o DataFrame
    conexao = pyodbc.connect(string_conexao_odbc)
    df = pd.read_sql(consulta_sql, conexao)
    conexao.close()
    
    # Atualiza o painel de pedidos
    pedidos_nao_separados = df[(df['cod_separacao'] == 2) | (df['cod_separacao'] == 8)]
    pedidos_ordenados = pedidos_nao_separados.sort_values(by=['data_emissao', 'data_entrega'])
    pedidos_painel = pedidos_ordenados.head(8)
    
    # Atualiza a visualização
    exibir_painel(janela, frame, pedidos_painel)

# Função para iniciar o painel
def iniciar_painel():
    # Cria a janela principal
    janela = tk.Tk()
    janela.title('Painel de Pedidos')

    # Define a janela para ser exibida maximizada
    janela.state('zoomed')

    # Cria um frame para conter os retângulos
    frame = tk.Frame(janela)
    frame.place(relx=0.5, rely=0.5, anchor='center')  # Centraliza o frame na janela

    # Inicia a atualização do painel
    atualizar_painel(janela, frame)

    # Exibe a janela
    janela.mainloop()

# Inicie o painel
iniciar_painel()


In [4]:
deve_continuar = False
